# Cleaning our Staging Data

### Introduction

So we just saw one way that we can clean up our staging models, and that is with the use of sources.  Now let's take some time to see some other ways we can clean up our staging layer.

#### 1. Properly identify

The first change we can make is simply to update our id.  If we look at our customers model, we can see that we do already have a `customer_id` for each record.

```sql
select * from customers limit 3;
```

<img src="./customer_id.png" width="100%">

But eventually, we'll be loading customers from other sources.  And then remember that at the integration layer, we'll be combining these customers into a single table.  So to identify that a given customer id refers to his id in `rds`, as opposed to say the customer information from hubspot, we can simply add a prefix saying rds to each of the ids like so. 

```sql
SELECT concat('rds-', customer_id) as customer_id
```

And adding this to our code, we can see that `rds-` is then prepended to each of our ids.

<img src="./update_customer_id.png" width="60%">

### 2. Normalizing Data

The next cleanup we can perform at the staging layer is simply to perform some basic normalization of our data.  For example, if we look at the original source data under our customers model, we can see that under customers, we also have information about the company a customer is associated with. 

<img src="./company_contacts.png" width="100%">

We can turn these into two separate tables: a customers and companies, and create an association of a customer belongs to a company.

Let's do that now.

We'll add a new file called `stg_rds_companies.sql`, and then we'll select the relevant company information.  

> Reminder: The company information is in the `customers` table, so that's why still begin by selecting from the customers source, and then pull out the relevant company columns.

> Write the following in the `stg_rds_companies.sql` file.

<img src="./add_companies.png" width="100%">

> You can view the results by running `dbt run` and then query the stg_rds_companies view.

So we have selected the relevant columns, but notice that we do not have an id for companies.  For that, we can simply use something that should be unique like the `company_name`.  We also can group by the `company_name` just in case there are any duplicates.

> Typo below -- it should be `max(address) as address`.

<img src="./company_name.png" width="100%">

> We are maxing every other column, because in postgres the select columns must either be the columns we are grouping by or an aggregate.

If we look at the company id below, we probably should provide some additional cleanup by replacing our spaces with hyphens, and by lowercasing the fullname.

```sql
concat('rds-',replace(lower(company_name), ' ', '-')) as company_id,
```

And now, if you run `dbt run` our company id is updated to be prepended with `rds-`:

`psql -d northwinds -c "select * from dev.stg_rds_customers order by company_id limit 3"`

> Your exact data will likely vary, but just make sure it is properly prepended.

<img src="./company-data.png">

With that in place, we can then associate a contact to it's related company through the company id.

To do that we should first run our staging models so that we create the new companies model.

`dbt run --models models/staging`

And from there, we can see that dbt created a new model called `stg_rds_companies`.

<img src="./stg_rds_companies.png" width="100%">

Finally, because we have a model where a customer `has_one` company, we'll want to associate the companies id to the related customer. 

We can do so with the following.

<img src="./customers-stg.png" width="60%">

> So above, we add a column for `company_id`, and we pull that `company_id` by first importing the stg_rds_companies model (where we generated the `company_id`) and joining it to our customers table.

And after calling `dbt run`, we can see that the company id is then added to the *customers* table.

> `psql -d northwinds -c "select * from dev.stg_rds_customers limit 1"`

<img src="./customer_company.png" width="100%">

Finally, let's add in the customer phone number, and format it as we did in the suppliers code.  

Our final code looks like the following:

```sql
WITH customers as (
  SELECT * FROM {{ source('rds', 'customers')}} 
), 
companies as (
  SELECT * FROM dev.stg_rds_companies
),
renamed as (
    SELECT 
    concat('rds-', customer_id) as customer_id, 
    SPLIT_PART(contact_name, ' ', 1) as first_name,
    SPLIT_PART(contact_name, ' ', -1) as last_name,
    REPLACE(TRANSLATE(phone, '(,),-,.', ''), ' ', '') as updated_phone,
    company_id
    FROM customers 
    JOIN companies ON companies.company_name = customers.company_name 
), final as 
(
  SELECT
   customer_id,
   first_name,
   last_name,
   CASE WHEN LENGTH(updated_phone) = 10 THEN
       '(' || SUBSTRING(updated_phone, 1, 3) || ') ' || 
       SUBSTRING(updated_phone, 4, 3) || '-' ||
       SUBSTRING(updated_phone, 7, 4) 
       END as phone,
   company_id
  FROM renamed
)
SELECT * FROM final
```

And in our initial results we get the following:

`psql -d northwinds -c "select * from dev.stg_rds_customers limit 3"`

<img src="./final-select.png" width="100%">

> Notice that we no longer have the country in the customers table, and that has been moved to the companies table.

### Summary

In this lesson, we saw two ways that we can clean up our staging models.  The first is adding a prefix to our ids to indicate the source.  And then we also separated some of our source data into multiple models.

To separate our data into multiple models, we needed to add an id for our new model of companies.  And then added the company_id to the customers model.